In [1]:
from rdflib import Graph

In [6]:
graph = Graph()
graph.parse("../data/external/page_ids_hu.ttl", format="ttl")

<Graph identifier=Ne8d46819ee494c63a9278b55e9533d8c (<class 'rdflib.graph.Graph'>)>

In [51]:
page_ids = pd.DataFrame([{
    "uri": str(uri),
    "uid": int(uid)
} for uri, _, uid in graph])#|.set_index("uri")

In [52]:
page_ids.head()

,uid,uri
0,1187577,http://hu.dbpedia.org/resource/Konplott
1,617297,http://hu.dbpedia.org/resource/Sablon:Népesség...
2,271910,http://hu.dbpedia.org/resource/Breinigerberg
3,907810,http://hu.dbpedia.org/resource/Kategória:1002
4,925842,http://hu.dbpedia.org/resource/Kategória:I._e....


In [53]:
import pandas as pd
from urllib.parse import unquote_plus

page_freqs = pd.read_csv("../data/external/page_inlinks.csv", sep=" ", header=None, engine="python")
page_freqs.columns=["freq", "uri"]
page_freqs.uri = page_freqs.uri.apply(unquote_plus)
# page_freqs.set_index("uri", inplace=True)

In [54]:
page_freqs.uri.values[0], page_ids.uri.values[0]

('http://hu.dbpedia.org/resource/Franciaország',
 'http://hu.dbpedia.org/resource/Konplott')

In [55]:
page_data = pd.merge(page_ids, page_freqs, how="inner", on=["uri"])

In [56]:
page_data.tail()

,uid,uri,freq
470452,836647,http://hu.dbpedia.org/resource/Efteling,4
470453,207142,"http://hu.dbpedia.org/resource/47,XYY_szindróma",2
470454,156708,http://hu.dbpedia.org/resource/RAF_(egyértelmű...,1
470455,1179840,http://hu.dbpedia.org/resource/Toszon,3
470456,117595,http://hu.dbpedia.org/resource/Kombinált_csűrő...,10


In [35]:
len(page_data), len(page_ids), len(page_freqs)

(736066, 736066, 2018820)

In [85]:
page_filter = (
    (page_data.freq>100) & 
    (~page_data.uri.str.contains("Kategória")) & 
    (~page_data.uri.str.contains("listája")) & 
    (~page_data.uri.str.contains("lista")) & 
    (~page_data.uri.str.contains("település")) & 
    (~page_data.uri.str.match(".*/.*\d+.*$"))
)

In [86]:
page_data[page_filter].sort_values(["freq"], ascending=False).to_csv("../data/interim/important_wikipages.csv")